# **[PolymerCost](https://www.polymercost.com)**
**by [OHPARC: The Ohio Polymer & Additives Research Center](https://www.ohparc.com)**

This Python Notebook will show our polymer formulation and costing software's core function spg_costs().  This core function is used to cost out any polymer formula with unlimited numbers of polymers and additives in any currency and in any units of measure.  All Software is MIT Licensed and available at www.github.com/OhioPolymerAdditivesResearchCenter/PolymerCost

# **Get the PolymerCost Python Code**

For security we are running this notebook with no need to connect to the internet at all.  There are no outside depdendencies to the PolymerCost code and no need to risk anything.  You can download this notebook and run it completely offline if you happen to be running it online.  If you are running it with your real compound data and pricing, you should probably be offline or understand that you are sharing that with google and your internet provider, not OHPARC.

First step is to get the code.  Running the cell below will import some of the PolymerCost code.  

In [ ]:
def spg_costs(formula):
    """takes formula, returns specific gravity and cost per pound and volume"""
    recipe = []
    sum_phr = sum_pound_volumes = sum_costs = 0
    for ingredient in formula:
        pound_volumes = ingredient[0] / ingredient[1]
        line_cost = ingredient[0] * ingredient[2]
        recipe_data = [ingredient[0], pound_volumes, line_cost]
        recipe.append(recipe_data)
    for item in recipe:
        sum_phr = sum_phr + item[0]
        sum_pound_volumes = sum_pound_volumes + item[1]
        sum_costs = sum_costs + item[2]
    formula_cost = sum_costs / sum_phr
    formula_spg = sum_phr / sum_pound_volumes
    volume_cost = formula_cost * formula_spg
    output = (formula_spg, formula_cost, volume_cost)
    return output

def flex_clear_dinp(phr):
    duro = 113.8 - 0.6*phr
    modulus = 3559.1 - 34.78*phr
    tensile = 4414.8 - 28.7*phr
    elongation = 248.6 + 1.61*phr
    clashberg = 14.23 - 0.76*phr
    brittle = -4.19 - 0.55*phr
    output = (duro, modulus, tensile, elongation, clashberg, brittle)
    return output

def flex_clear_dinch(phr):
    duro = 115.2 - 0.602*phr
    modulus = 3124.3-28.3*phr
    tensile = 4274.7-26.3*phr
    elongation = 335.1+.6*phr
    clashberg = 11.64-.79*phr
    brittle = -6.71-.57*phr
    output = (duro, modulus, tensile, elongation, clashberg, brittle)
    return output

def flex_clear_dotp(phr):
    duro = 113.8 - 0.617*phr
    modulus = 3650.3-35.4*phr
    tensile = 4419.7-26.8*phr
    elongation = 250.9+1.96*phr
    clashberg = 10.8-.77*phr
    brittle = -4.7-.57*phr
    output = (duro, modulus, tensile, elongation, clashberg, brittle)
    return output

def flex_filled(duro_A, phr_CaCO3):
    #linear r square 0.989, n = 27
    #duro = 0.929607437 * duro_A + 0.0531305301 * phr_CaCO3 + 6.270608278
    #polynomial r square 0.992, n=27
    a = -0.001058114988 * duro_A * duro_A
    b = -0.001695411881 * duro_A * phr_CaCO3
    c = 0.00004967760719 * phr_CaCO3 * phr_CaCO3
    d = 1.158214659 * duro_A
    e = 0.1840327098 * phr_CaCO3
    f = -5.05711084
    duro = a + b + c + d + e + f
    return duro

def duro_AtoD(duro_A):
    #converter {k:v, A Shore:D Shore, another set of A/D, so on}
    converter = {120:84, 115:84, 110:75.8, 105:66.8,
                 100:58, 95:46, 90:39, 85:33, 80:29, 75:25, 70:22,
                 65:19, 60:16, 55:14, 50:12, 45:10, 40:8, 35:7, 30:6}
    test_A = (duro_A // 5)*5
    if test_A >= 118.2:
        output = 84
    elif test_A <=30:
        output = 6
    else:
        total_gap_up = converter[test_A + 5] - converter[test_A]
        share_gap_up = (duro_A - test_A)/5
        partial_adjust = total_gap_up * share_gap_up
        output = converter[test_A] + partial_adjust
    return output

# **Flexible PVC Compounder Simulation**

Our simulation is a fictional compounder of flexible PVC products.  They make their own compound and produce finished goods from that compound.  That is 80% of their business, the other 20% is them producing pellets for outside customers with their spare capacity to help pay for the equipment.

They are a Formosa house and use their product exclusively.  They have railcars on site with Formosa Formolon 622F, Formolon 676A, and Formolon 680A in stock at all times.

They have considerable bulk tank storage for plasticizer as a flexible producer.  They have dedicated tanks for BASF products Palatinol DINP, Platinol DOTP, and Hexamoll DINCH.  They also have a dedicated tank for ExxonMobil's Jayflex DINP.

Filler is also provided by rail.  They use Piqua Materials PIQ-White Ultra Premium Filler 30, a 3 micron Calcium Carbonate (CaCO3) filler.

Being a fictional compounder, they like to keep it simple so they receive their Heat Stabilizer, Epoxidized Soybean Oil, and various lubricants in a customer specific liquid one pack from Baerlocher with a BAEROPOL brand name.

They have additional colorants in stock from Akronchem and Lanxess that are dosed at the machine after the base compound is made as those colors are needed.

So here is a list of the products we have in house to work with, minus the colors which are used at the press.

**PVC Resin**

Formolon 622F, 676A, and 680A.  All three have a Specific Gravity or spg of 1.40.  Fictional pricing is \$0.405, \$0.410, and \$0.435 each.

**Plasticizer**

BASF Platinol DINP and DOTP and Hexamoll DINCH.  DINP is 0.975 spg and has a fictional price of \$0.765 per pound.  DOTP is 0.984 spg and has a price of \$0.765 per pound also.  BASF is really pushing the DINCH.  They have that at 0.949 spg and are selling it to us at $0.735 per pound.

They also have a tank of ExxonMobil's DINP.  It has a spg of 0.973 and a fictional price of \$0.735.

**Stabilizer / ESO / Lubricants**

They use Baerlocher here for these products exclusively.  One option for their fictional product line fits all.  The BAEROPOL is at 1.015 spg and costs them $0.935 per pound.

**Filler**

Again they use one material exclusively, the Piqua Materials 3 micron white thanks to its consistency of cut and color.  This product is at 2.71 spg and is costing them only \$0.095 per pound.

**Formulas**

This company is producing flexible PVC for its internal production that is later colored at the press when parts are made. 

**The Base Formula**

PVC = Formolon 622F @ 100 PHR @ 1.40 spg @ \$0.405 per pound

Plasticizer = ExxonMobil's DINP @ 60 PHR @ 0.973 spg @ \$0.735 per pound

Stabilizer = Baerlocher's BAEROPOL @ 7.5 PHR @ 1.015 spg @ \$0.935 per pound

Filler = Piqua's 3 micron white CaCO3 @ 10 PHR @ 2.71 @ \$0.095 per pound.

First, lets write that formula in code.







In [ ]:
current_formula = [(100, 1.40, 0.405),
                   (60, 0.973, 0.735),
                   (7.5, 1.015, 0.935),
                   (10, 2.71, 0.095)]

# **Run spg_costs() on our current_formula**

First let's run our spg_costs() on the current formula.

In [ ]:
spg_costs(current_formula)

(1.2311621386153346, 0.5214788732394366, 0.6420250448201799)

This calculates our specific gravity to be approximately 1.231, our material cost is \$0.52148 per pound, and \$0.64203 per volume.

Now let's explore the physical properties of that compound.

# **Plasticizing PVC**

The next step after the spg and costs, is the plasticization.  PolymerCost includes functions for many common PVC plasticizers on the market.  They are all in the code as flex_clear_PLASTICIZER_NAME().  In our current_formula we were using DINP so we will use the function flex_clear_dinp(PHR) to plasticize the resin.  We just need to pass the variable or parameter PHR to the function.  In current_formula, they are using 60PHR so let's run that.

In [ ]:
flex_clear_dinp(60)

(77.8, 1472.2999999999997, 2692.8, 345.2, -31.37, -37.19)

Running the flex_clear_dinp() returned 6 results.  

1.  77.8 A Shore Durometer per ASTM D2240 @ 15s delay
2.  1472 psi Modulus @ 100% Elongation per ASTM D638
3.  2693 psi Tensile Strenght per ASTM D638
4.  345% Ultimate Elongation per ASTM D638
5.  -31.4 Degrees Celsius Clash-Berg per ASTM D1043
6.  -37.2 Degrees Celsius Brittleness per ASTM D746

Those are the estimated physical properties of a PVC clear compound plasticized with DINP at 60 PHR.  

## **Filling Plasticized PVC**

In [ ]:
flex_filled(77.8, 10)

79.17365332153508

Their formula wasn't clear, it used 10 PHR of Piqua's 3 micron CaCO3 filler to give it a white base to later color.



PolymerCost can calculate for the addition of filler on the hardness or durometer reading. 

 To compensate for this I ran the flex_filled(durometer, PHR filler) function.  
 
 To run it you must pass two variables or parameters, the durometer of the compound if it were clear from the previous function, and the amount of filler you want to compensate for.  The result is the calculated A Shore Durometer @ 15 second delay per ASTM D2240 for the filled compound.  
 
 It is not a straight line and there is a twist for each plasticizer.  You can see that graph for DINP and 3 micron filler as produced by the PolymerCost software [here](https://www.ohparc.com/img/spinner.gif).

 Our results here showed the durometer being 79.2 A Shore, increased from 77.8 if it were the same formula but without filler.

 PolymerCost lacks sufficient data to modify the properties other than cost, spg, and hardness in flexible PVC.  Generally mineral fillers make the compound stiffer but more brittle but the data is too scattered to be reliable with different filler particle sizes and resin IV's not disclosed in some of the data used.

 Fortunately, cost, spg, and hardness are by far the 3 most important factors in flexible PVC, and probably why there is data there.  The fictional compounder feels that as long as the cost is good and the hardness is the same their parts should perform just fine.
 



# **New Formula Trials**

Let's build some formulas and see if maybe we can't save some money.  We are currently making 50 million pounds of our current formula so we don't need to find much to have it add up to a large amount.

The current formula is at 79.2 hardness.  That number and cost are apparently our only concerns in this fictional scenario.

Well if adding the filler made it harder and the plasticizer made it softer, maybe we just don't add the filler and use less plasticizer?  Use none of one product and save all that money and use less of the other.  Sounds like a win win.

We can run it through PolymerCost and see how we do.

In [16]:
no_filler_trial = [(100, 1.40, 0.405),
                   (57.7, 0.973, 0.735),
                   (7.5, 1.015, 0.935)]

In [17]:
spg_costs(no_filler_trial)

(1.1960712287994253, 0.5443220338983051, 0.6510479239473482)

In [18]:
flex_clear_dinp(57.70)

(79.18,
 1552.2939999999996,
 2758.8100000000004,
 341.497,
 -29.622000000000003,
 -35.925000000000004)

# **No Filler Trial Results**

Both formulas were at 79.2 A Shore Durometer so they were a match as far as production would be concerned.  

The no filler trial ended up being more expensive though on a cost per pound and cost per volume basis.  Worse yet, they need at least 5 PHR filler in there to give it a cloudy white base to color, otherwise they need to use more expensive Titanium Dioxide at the press. 

It turns out that the same formula with less plasticizer and no filler wasn't cheaper at all.  Sounds counter intuitive somewhat to take out an ingredient completely and remove another and not save money, but that is the science of compounding.  The chemical you took out was the cheapest volume chemical in the formula, despite being the heaviest.

So back to the drawing board, this time we ask Engineering what is the most filler we can put in a formula as accountants.  He says you can't put any in the machines, but you can use up to 50 PHR in your spreadsheets and software and our equipment can run the formula.

This trial was a horrible idea.  At least PolymerCost proved that out before spending any labor, machine time, or materials on it.

## **So let's try a high filler formula with Polymer Cost.**

In [29]:
high_filler_trial = [(100, 1.40, 0.405),
                   (64, 0.973, 0.735),
                   (7.5, 1.015, 0.935),
                   (50, 2.71, 0.095)]

In [30]:
spg_costs(high_filler_trial)

(1.358530076237302, 0.4483182844243792, 0.6090538731176284)

In [31]:
flex_clear_dinp(64)

(75.4, 1333.1799999999998, 2578.0, 351.64, -34.41, -39.39)

In [32]:
flex_filled(75.4, 50)

79.19084816002692

# **High Filler Trial Results**

Here we again matched the hardness specification of 79.2 A Shore @ 15 second delay.  This time we are at \$0.448 per pound and \$0.609 per volume.

Our current compound in production is at \$0.521 per pound and \$0.642 per volume.  

This formula has potential and should work per Engineering.  Let's double check the spreadsheet math and run this down to Finance and Engineering, we may have something here.

Checking the math, you can ignore the cost per pound figure as our parts will weigh more now with an increased specific gravity.  On a volume basis we are 5.1% cheaper though and that will be our materials savings from this formula.

Congratulations you just saved \$1.32 million on an annual spend of \$26 million through proper formulation.  Nice day in the office.

Let's take this back to Engineering and bring Finance with us.





# **More Trials, Plasticizer and Filler now**

So just jacking up the filler to the Engineering max worked, we just had to add some plasticizer to the equation to compensate.

Well we have one more variable in house to play with, our plasticizer tank farm.  We are using the cheapest DINP we can get, but what about that cheaper DINCH, we have to use more of it, but it weighs less than DINP.  What about that DOTP or the BASF version of ExxonMobil's DINP any potential there?  Those don't look as promising to me for this cost centric application, so let's just focus on the DINCH says Engineering.

First we need to make the formula.

In [77]:
dinch_high_filler_trial = [(100, 1.40, 0.405),
                   (66.25, 0.949, 0.735),
                   (7.5, 1.015, 0.935),
                   (50, 2.71, 0.095)]

In [78]:
spg_costs(dinch_high_filler_trial)

(1.3391929020856792, 0.45120111731843576, 0.604245333725977)

In [79]:
flex_clear_dinch(66.25)

(75.3175, 1249.4250000000002, 2532.325, 374.85, -40.697500000000005, -44.4725)

In [80]:
flex_filled(75.3, 50)

79.09944954640108

# **DINCH HIGH FILLER TRIAL**

This trial was even better than the other.  The lower efficiency DINCH means I had to use more of it than DINP, but it is lighter than both DINP and the resin it replaces so it was a cost savings.  What was \$1.32 million in annual savings is now a cool \$1.5 million in savings every year.  Thanks PolymerCost and OHPARC, we will be running this trial ASAP.

The calculations for the BASF DINP and DOTP weren't economically feasible, only there DINCH is competitive with ExxonMobil's DINP with these fictional prices.


